# Using Db2 11.5.4 RESTful Programming with Data Virtualization

The following notebook is a brief example of how to use the Db2 11.5.4 RESTful Endpoint service to extend the capabilies of Cloud Pak for Data Virtualization. 

You can extend your Cloud Pak for Data system so that application programmers can create Representational State Transfer (REST) endpoints that can be used to interact with the Data Virtualization Service. 

Each endpoint is associated with a single SQL statement. Authenticated users of web, mobile, or cloud applications can use these REST endpoints from any REST HTTP client without having to install any Db2 drivers.

The Db2 REST server accepts an HTTP request, processes the request body, and returns results in JavaScript Object Notation (JSON).

The Db2 REST server is pre-installed and running on Docker on server7 (10.1.1.12) in the Demonstration cluster. As a programmer you can communicate with the service on port 50050. Your welcome note includes the external port you can use to interact with the Db2 RESTful Endpoint service directly.

You can find more information about this service at: https://www.ibm.com/support/producthub/db2/docs/content/SSEPGG_11.5.0/com.ibm.db2.luw.admin.rest.doc/doc/c_rest.html.

## Finding the Db2 RESTful Endpoint Service API Documentation
If you are running this notebook from a browser running inside the Cloud Pak for Data cluster, click: http://10.1.1.12:50050/docs
If you are running this from a browser from your own desktop, check your welcome note for the address of the Db2 RESTful Service at port 50050.

## Import the required programming libraries
The requests library is the minimum required by Python to construct RESTful service calls. The Pandas library is used to format and manipulate JSON result sets as tables. 

In [ ]:
import requests
import pandas as pd

## Create the Header File required for getting an authetication token
The RESTful call to the Db2 RESTful Endpoint service is contructed and transmitted as JSON. The first part of the JSON structure is the headers that define the content tyoe of the request.

In [ ]:
headers = {
  "content-type": "application/json"
}

## RESTful Host
The next part defines where the request is sent to. It provides the location of the RESTful service for our calls.

In [ ]:
Db2RESTful = "http://10.1.1.12:50050"

## API Authentication Service
Each service has its own path in the RESTful call. For authentication we need to point to the `v1/auth` service.

In [ ]:
API_Auth = "/v1/auth"

## Authentication
To authenticate to the RESTful service you must provide the connection information for the database along with the userid and password that you are using to authenticate with. You can also provide an expiry time so that the access token that gets returned will be invalidated after that time period.

In [ ]:
body = {
  "dbParms": {
    "dbHost": "10.1.1.1",
    "dbName": "bigsql",
    "dbPort": 30797,
    "isSSLConnection": False,
    "username": "user999",
    "password": "cEs*F5nUZ32#5?_w"
  },
  "expiryTime": "300m"
}

## API Service
When communicating with the RESTful service, you must provide the name of the service that you want to interact with. In this case the authentication service is */v1/auth*. 

In [ ]:
try:
    response = requests.post("{}{}".format(Db2RESTful,API_Auth), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

A response code of 200 means that the authentication worked properly, otherwise the error that was generated is printed.

In [ ]:
print(response)

The response includes a connection token that is reused throughout the rest of this lab. It ensures secure a connection without requiring that you reenter a userid and password with each request.  

In [ ]:
if (response.status_code == 200):
  token = response.json()["token"]
  print("Token: {}".format(token))
else: 
  print(response.json()["errors"])

## Reusing the token in the standard header
The standard header for all subsequent calls will use this format. It includes the access token.

In [ ]:
headers = {
  "authorization": f"{token}",
  "content-type": "application/json"
}

## Executing an SQL Statement
Executing SQL requires a different service endpoint. In this case we will use "/services/execsql"

In [ ]:
API_execsql = "/v1/services/execsql"

In this example, the code requests that the RESTful function waits until the command is complete.

In [ ]:
body = {
  "isQuery": True,
  "sqlStatement": "SELECT * FROM STOCK.STOCK_SYMBOLS WHERE SYMBOL = 'CAT'",
  "sync": True
}

In [ ]:
try:
    response = requests.post("{}{}".format(Db2RESTful,API_execsql), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
print(response)

Retrieve the results. The Dataframe class converts the json result set into a table. Dataframes can be used to further manipulate results in Python.

In [ ]:
display(pd.DataFrame(response.json()['resultSet']))

## Use Parameters in a SQL Statement
Simple parameter passing is also available through the execsql service. In this case we are passing the Stock symbol "CAT" into the query to retrieve the full stock name. Try substituting different symbols and run the REST call again. Symbols like PG, DIS, or MMM.

In [ ]:
body = {
  "isQuery": True,
  "parameters" : {
      "1" : "CAT"
  },
  "sqlStatement": "SELECT * FROM STOCK.STOCK_SYMBOLS WHERE SYMBOL = ?",
  "sync": True
}

In [ ]:
try:
    response = requests.post("{}{}".format(Db2RESTful,API_execsql), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
response.json()["resultSet"]
display(pd.DataFrame(response.json()['resultSet']))

## Generate a Call and don't wait for the results
Turn sync off to require us to poll for the results. This is especially useful for long running queries. 

In [ ]:
body = {
  "isQuery": True,
  "sqlStatement": "SELECT * FROM STOCK.STOCK_SYMBOLS WHERE SYMBOL = 'CAT'",
  "sync": False
}

In [ ]:
try:
    response = requests.post("{}{}".format(Db2RESTful,API_execsql), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
print(response)

Retrieve the job id to retrieve the results later.

In [ ]:
job_id = response.json()["id"]

In [ ]:
print(job_id)

## Retrieve Result set using Job ID
The service API needs to be appended with the Job ID.

In [ ]:
API_get = "/v1/services/"

We can limit the number of rows that we return at a time. Setting the limit to zero means all of the rows are to be returned.

In [ ]:
body = {
  "limit": 0
}

Get the results.

In [ ]:
try:
    response = requests.get("{}{}{}".format(Db2RESTful,API_get,job_id), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

Retrieve the results.

In [ ]:
print(response.json()["resultSet"])
display(pd.DataFrame(response.json()["resultSet"]))

## Create a Unique RESTful Service
The most common way of interacting with the service is to fully encapsulate an SQL statement, including any parameters, in a unique RESTful service. This creates a secure separation between the database service and the RESTful programming service. It also allows you to create versions of the same service to make maintenance and evolution of programming models simple and predictable. 

In [ ]:
API_makerest = "/v1/services"

Define the SQL that we want in the RESTful call.

In [ ]:
body = {"isQuery": True,
       "parameters": [
         {
         "datatype": "CHAR(4)",
         "name": "@SYMBOL"
         }
       ],
       "schema": "STOCK",
       "serviceDescription": "Get full name given symbol",
       "serviceName": "getstock",
       "sqlStatement": "SELECT * FROM STOCK.STOCK_SYMBOLS WHERE SYMBOL = @SYMBOL",
       "version": "1.0"
}

In [ ]:
try:
    response = requests.post("{}{}".format(Db2RESTful,API_makerest), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
if (response.status_code == 201):
  print("Service Created")
else:
  print(response.json()['errors'])

## Call the new RESTful Service
Now you can call the RESTful service. In this case we will pass the stock symbol CAT. But like in the previous example you can try rerunning the service call with different stock symbols.

In [ ]:
API_runrest = "/v1/services/getstock/1.0"

In [ ]:
body = {
  "parameters": {
    "@SYMBOL": "CAT"
  },
  "sync": True
}

In [ ]:
try:
    response = requests.post("{}{}".format(Db2RESTful,API_runrest), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

A response of 200 indicates a successful service call.

In [ ]:
print(response)

You can now retrieve the result set, convert it into a Dataframe and display the table.

In [ ]:
print(response.json())
display(pd.DataFrame(response.json()['resultSet']))

## Retreive Service Details
You can query each service to see its details, including authoritization, input parameters and output results. 

In [ ]:
API_listrest = "/v1/services/getstock/1.0"

In [ ]:
try:
    response = requests.get("{}{}".format(Db2RESTful,API_listrest), headers=headers)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
print(response.json())

In [ ]:
print("Service Details:")
print("Service Name: " + response.json()['serviceName'])
print("Service Version: " + response.json()['version'])
print("Service Description: " + response.json()['serviceDescription'])
print("Service Creator: " + response.json()['serviceCreator'])
print("Service Updater: " + response.json()['serviceUpdater'])


print('Users:')
display(pd.DataFrame(response.json()['grantees']['users']))
print('Groups:')
display(pd.DataFrame(response.json()['grantees']['groups']))
print('Roles:')
display(pd.DataFrame(response.json()['grantees']['roles']))

print('')
print('Input Parameters:')
display(pd.DataFrame(response.json()['inputParameters']))

print('Result Set Fields:')
display(pd.DataFrame(response.json()['resultSetFields']))



## List Available Services
You can also list all the user defined services you have access to

In [ ]:
API_listrest = "/v1/services"

In [ ]:
try:
    response = requests.get("{}{}".format(Db2RESTful,API_listrest), headers=headers)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
display(pd.DataFrame(response.json()['Db2Services']))

## Delete a Service
A single call is also available to delete a service

In [ ]:
API_deleteService = "/v1/services"
Service = "/getstock"
Version = "/1.0"

In [ ]:
try:
    response = requests.delete("{}{}{}{}".format(Db2RESTful,API_deleteService,Service,Version), headers=headers)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
print (response)

## Get Service Logs - Coming Soon
You can easily download service logs. However you must be authorized as the principal administration user to do so.

In [ ]:
API_listrest = "/v1/logs"

In [ ]:
try:
    response = requests.get("{}{}".format(Db2RESTful,API_listrest), headers=headers)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
if (response.status_code == 200):
  myFile = response.content
  open('/tmp/logs.zip', 'wb').write(myFile)
  print("Downloaded",len(myFile),"bytes.")
else:
  print(response.json())